In [1]:
import numpy as np
import tensorflow as tf
import sys
import time
from datetime import timedelta
import tensorflow.contrib.keras as kr
from sklearn import metrics
from sklearn.model_selection import KFold

import moxing as mox
mox.file.shift('os', 'mox')

INFO:root:Using MoXing-v1.14.1-ddfd6c9a
INFO:root:Using OBS-Python-SDK-3.1.2


In [2]:
trainDataPath = "s3://corpus-text-classification1/data/train_5500.label.txt"
testDataPath = "s3://corpus-text-classification1/data/TREC_10.label.txt"
vocabPath = "s3://corpus-text-classification1/data/glove.6B.100d.txt"

In [3]:
def readfile(filePath):
    """读取文件内容，返回文本和标签列表"""
    train_data = []
    with open(filePath, 'r', encoding='utf-8', errors='ignore') as f:
        for line in f.readlines():
            word = line.strip().split()
            label = word[0].split(":")[0]
            content = word[1:]
            train_data.append([content,label])
    
    np.random.shuffle(train_data)
    return np.asarray(train_data)


def loadGloVe(filename):
    vocab = []
    embd = []
    print('Loading GloVe!')
    # vocab.append('unk') #装载不认识的词
    # embd.append([0] * emb_size) #这个emb_size可能需要指定
    file = open(filename,'r',encoding='utf-8')
    for line in file.readlines():
        row = line.strip().split(' ')
        vocab.append(row[0])
        embd.append([float(ei) for ei in row[1:]])
    file.close()
    print('Completed!')
    return vocab,embd


def process_file(contents, labels, word_to_id, cat_to_id, num_classes, pad_max_length):
    """
    将文件转换为id表示,并且将每个单独的样本长度固定为pad_max_lengtn
    """
    # contents, labels = readfile(filePath)
    data_id, label_id = [], []
    # 将文本内容转换为对应的id形式
    for i in range(len(contents)):
        data_id.append([word_to_id[x] for x in contents[i] if x in word_to_id])
        label_id.append(cat_to_id[labels[i]])
    # 使用keras提供的pad_sequences来将文本pad为固定长度
    x_pad = kr.preprocessing.sequence.pad_sequences(data_id, pad_max_length)
    ''' https://blog.csdn.net/TH_NUM/article/details/80904900
    pad_sequences(sequences, maxlen=None, dtype=’int32’, padding=’pre’, truncating=’pre’, value=0.) 
        sequences：浮点数或整数构成的两层嵌套列表
        maxlen：None或整数，为序列的最大长度。大于此长度的序列将被截短，小于此长度的序列将在后部填0.
        dtype：返回的numpy array的数据类型
        padding：‘pre’或‘post’，确定当需要补0时，在序列的起始还是结尾补
        truncating：‘pre’或‘post’，确定当需要截断序列时，从起始还是结尾截断
        value：浮点数，此值将在填充时代替默认的填充值0
    '''
    y_pad = kr.utils.to_categorical(label_id, num_classes=num_classes)  # 将标签转换为one-hot表示
    ''' https://blog.csdn.net/nima1994/article/details/82468965
    to_categorical(y, num_classes=None, dtype='float32')
        将整型标签转为onehot。y为int数组，num_classes为标签类别总数，大于max(y)（标签从0开始的）。
        返回：如果num_classes=None，返回len(y) * [max(y)+1]（维度，m*n表示m行n列矩阵，下同），否则为len(y) * num_classes。
    '''
    return x_pad, y_pad

In [4]:
categories = ['ABBR', 'DESC', 'ENTY', 'HUM', 'LOC', 'NUM']
num_classes = len(categories)

cat_to_id = {'ABBR': 0, 'DESC': 1, 'ENTY': 2, 'HUM': 3, 'LOC': 4, 'NUM': 5}

vocab, embd = loadGloVe(vocabPath)
vocab_size = len(vocab)
embedding_dim = len(embd[0])
embedding = np.asarray(embd)
word_to_id = dict(zip(vocab, range(vocab_size)))

print(len(embedding),embedding_dim,vocab_size)

testData = readfile(testDataPath)
trainData = readfile(trainDataPath)

print(len(testData),len(trainData))
trainData = np.r_[trainData,testData]
np.random.shuffle(trainData)
len(trainData)

seq_length = 37

Loading GloVe!
Completed!
400000 100 400000
500 5452


In [5]:
def train_10_fold(train_data, categories):
    
    tx, ty = process_file(train_data[:,0], train_data[:,1], word_to_id, cat_to_id, num_classes, seq_length)
    print(len(tx),len(tx[0]),len(tx[1]))
    
    fold_id = 0
    test_acc = []
    
    kf = KFold(n_splits=10)
    for train_i, test_i in kf.split(tx):
        fold_id += 1
        print("Fold: ", fold_id)
        test_acc.append(classifier.train(
            X_train=tx[train_i],
            y_train=ty[train_i],
            X_eval=tx[test_i],
            y_eval=ty[test_i],
            categories=categories,
            epochs=30
        ))
    print(test_acc)
    print("%s, %s, %s, %s" % (np.mean(test_acc),np.std(test_acc),np.std(test_acc,ddof=1),np.var(test_acc)))
    return test_acc

In [6]:
class Classifier:

    def __init__(self, model, input_length, output_length):
        self.model = model
        self.input_length = input_length
        self.output_length = output_length

    def compile(self, batch_size=32):
        self._ds_x = tf.placeholder(tf.float32, [None, self.input_length])
        self._ds_y = tf.placeholder(tf.float32, [None, self.output_length])

        ds = tf.data.Dataset.from_tensor_slices((self._ds_x, self._ds_y))
        ds = ds.batch(batch_size)

        self._ds_it = ds.make_initializable_iterator()
        self._input, self._labels = self._ds_it.get_next()

        self._features = self.model(self._input)
        self._output = _create_dense_layer(self._features, self.output_length)

        self._create_acc_computations()
        self._create_backpropagation()

    def _create_acc_computations(self):
        self._predictions = tf.argmax(self._output, 1)
        labels = tf.argmax(self._labels, 1)
        self._accuracy = tf.reduce_mean(
            tf.cast(tf.equal(self._predictions, labels), 'float32'))

    def _create_backpropagation(self):
        losses = tf.nn.softmax_cross_entropy_with_logits_v2(
            logits=self._output,
            labels=self._labels)
        self._loss = tf.reduce_mean(losses)

        optimizer = tf.train.AdamOptimizer(0.001)
        global_step = tf.Variable(0, name="global_step", trainable=False)
        grads_and_vars = optimizer.compute_gradients(self._loss)

        self._train_op = optimizer.apply_gradients(
            grads_and_vars, global_step=global_step)

    def summary(self):
        print('input:', self._input.shape)
        self.model.summary()
        print('output:', self._output.shape)

    def train(self, X_train, y_train, X_eval, y_eval, categories, epochs=20, require_improve=3):
        
        session = tf.Session()
        session.run(tf.global_variables_initializer())
        session.run(tf.local_variables_initializer())
        
        best_vac_acc = 0.0
        last_improved = 0
        
        for e in range(epochs):
            start_time = time.time()
            loss, acc = self._train(X_train, y_train, session)
            duration = time.time() - start_time

            val_loss, val_acc = self._eval(X_eval, y_eval, session)
            
            if val_acc > best_vac_acc:
                best_vac_acc = val_acc
                last_improved = e
                improved_str = '*'
            else:
                improved_str = ''
            
            output = 'Epoch: {:>1}, Train Loss: {:>6.4}, Train Acc: {:>6.2%}, Val Loss: {:>6.4}, Val Acc: {:>6.2%}, Time: {:.2f}s {}'
            print(output.format(e + 1, loss, acc, val_loss, val_acc, duration, improved_str))
            
            if e - last_improved > require_improve:
                print("No optimization for a long time, auto-stopping...")
                
                y_test_cls = np.argmax(y_eval, 1)  # 获得类别
                y_test_pred_cls = np.argmax(self.predict(X_eval, session), 1)
                accuracy_score = metrics.accuracy_score(y_test_cls, y_test_pred_cls)
                
                # evaluate
                print("Precision, Recall and F1-Score...")
                print(metrics.classification_report(y_test_cls, y_test_pred_cls, target_names=categories))
                '''
                sklearn中的classification_report函数用于显示主要分类指标的文本报告．在报告中显示每个类的精确度，召回率，F1值等信息。
                    y_true：1维数组，或标签指示器数组/稀疏矩阵，目标值。 
                    y_pred：1维数组，或标签指示器数组/稀疏矩阵，分类器返回的估计值。 
                    labels：array，shape = [n_labels]，报表中包含的标签索引的可选列表。 
                    target_names：字符串列表，与标签匹配的可选显示名称（相同顺序）。 
                    原文链接：https://blog.csdn.net/akadiao/article/details/78788864
                '''

                print("Confusion Matrix...")
                print(metrics.confusion_matrix(y_test_cls, y_test_pred_cls))
                '''
                混淆矩阵是机器学习中总结分类模型预测结果的情形分析表，以矩阵形式将数据集中的记录按照真实的类别与分类模型作出的分类判断两个标准进行汇总。
                这个名字来源于它可以非常容易的表明多个类别是否有混淆（也就是一个class被预测成另一个class）
                https://blog.csdn.net/u011734144/article/details/80277225
                '''
                break
        # endfor
        session.close()
        return accuracy_score

    def _train(self, X_train, y_train, session):
        import numpy as np

        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_train,
                self._ds_y: y_train
            })
        loss, acc, = [], []
        while True:
            try:
                _, vloss, vacc = session.run(
                    fetches=[self._train_op, self._loss, self._accuracy])

                loss.append(vloss)
                acc.append(vacc)
            except tf.errors.OutOfRangeError:
                break
        # endwhile

        loss, acc = np.mean(loss), np.mean(acc)
        return loss, acc

    def _eval(self, X_val, y_val, session):
        session.run(
            fetches=self._ds_it.initializer,
            feed_dict={
                self._ds_x: X_val,
                self._ds_y: y_val
            })

        loss, acc, = 0, 0
        while True:
            try:
                l, vloss, vacc = session.run(
                    fetches=[self._labels, self._loss, self._accuracy])

                loss += vloss * len(l)
                acc += vacc * len(l)
            except tf.errors.OutOfRangeError:
                break

        return loss / len(X_val), acc / len(X_val)

    def predict(self, X, session):
        

        session.run(self._ds_it.initializer,
                         feed_dict={
                             self._ds_x: X,
                             self._ds_y: np.empty((len(X), self.output_length))
                         }
                         )

        pred = list()
        while True:
            try:
                ppred = session.run(tf.nn.softmax(self._output))

                pred.extend(map(lambda l: l.tolist(), ppred))
            except tf.errors.OutOfRangeError:
                break

        return pred

def _create_dense_layer(x, output_length):
    '''Creates a dense layer
    '''
    input_size = x.shape[1].value
    W = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[input_size, output_length],
            stddev=0.1))
    b = tf.Variable(
        initial_value=tf.truncated_normal(
            shape=[output_length]))

    dense = tf.nn.xw_plus_b(x, W, b)

    return dense

In [7]:
class ShinContextualModel:
    '''
    Implementation proposal of: http://milab.snu.ac.kr/pub/BigComp2018.pdf
    '''
    def __init__(self, embedding, conv_size=3, num_layers=2):
        '''Constructor.
        # Parameters:
        embedding: numpy array representing the embedding.
        conv_size: Size of the convolutions. Number of words that takes each
            convolution step.
        num_layers: Number of recurrent convolutions.
        '''
        self._embedding = embedding
        self._conv_size = conv_size
        self._num_layers = num_layers

    def __call__(self, input):
        self._embedding_tf = self._create_embedding_layer(
            self._embedding, input)
        self._convolution_tf = self._create_convolutional_layers(
            self._conv_size, self._num_layers, self._embedding_tf)
        self._pooling_tf = self._create_maxpooling_layer(self._convolution_tf)

        return self._pooling_tf

    def summary(self):
        print('embedding:', str(self._embedding_tf.shape))
        print('conv:', str(self._convolution_tf.shape))
        print('pool:', str(self._pooling_tf.shape))

    def _create_embedding_layer(self, embedding, input_x):
        embedding = tf.Variable(initial_value=embedding)

        embedded_chars = tf.nn.embedding_lookup(
            embedding, tf.cast(input_x, 'int32'))

        return embedded_chars

    def _create_convolutional_layers(self, conv_size, num_layers, embedding):
        filter_height = conv_size
        filter_width = embedding.shape[2].value

        filter_shape = [filter_height, filter_width, filter_width]

        W = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=filter_shape,
                stddev=0.1))
        b = tf.Variable(
            initial_value=tf.truncated_normal(
                shape=[filter_width]))

        z = embedding
        for _ in range(num_layers):
            conv = tf.nn.conv1d(
                value=z,
                filters=W,
                stride=1,
                padding='SAME')
            bias = tf.nn.bias_add(conv, b)
            c = tf.nn.relu(bias)

            d = tf.nn.dropout(c, 0.75)
            # Add BatchNormalization or LocalResponseNormalization
            e = tf.expand_dims(d, 1)

            z = tf.nn.local_response_normalization(
                e,
                depth_radius=5,
                bias=1,
                alpha=0.001,
                beta=0.75
            )
            z = tf.squeeze(z, 1)
        # endfor
        return z

    def _create_maxpooling_layer(self, convolution):
        conv_size = convolution.shape[1].value
        embedding_size = convolution.shape[2].value

        convolution = tf.expand_dims(convolution, -1)
        pooled = tf.nn.max_pool(
            value=convolution,
            ksize=[1, conv_size, 1, 1],
            strides=[1, 1, 1, 1],
            padding='VALID')

        flat = tf.reshape(pooled, [-1, embedding_size])
        return flat

In [8]:
word_vector = embedding.astype('float32')
model = ShinContextualModel(embedding=word_vector)

classifier = Classifier(
    model=model,
    input_length=seq_length,
    output_length=num_classes)

classifier.compile(batch_size=32)
classifier.summary()

input: (?, 37)
embedding: (?, 37, 100)
conv: (?, 37, 100)
pool: (?, 100)
output: (?, 6)


In [9]:
train_10_fold(trainData, categories)

5952 37 37
Fold:  1
Epoch: 1, Train Loss:  1.666, Train Acc: 33.35%, Val Loss:  1.346, Val Acc: 46.31%, Time: 18.20s *
Epoch: 2, Train Loss:  1.173, Train Acc: 55.38%, Val Loss:   1.04, Val Acc: 60.23%, Time: 17.22s *
Epoch: 3, Train Loss: 0.8738, Train Acc: 67.99%, Val Loss: 0.8806, Val Acc: 67.79%, Time: 16.89s *
Epoch: 4, Train Loss: 0.6611, Train Acc: 76.75%, Val Loss: 0.8114, Val Acc: 71.64%, Time: 16.99s *
Epoch: 5, Train Loss:  0.548, Train Acc: 80.52%, Val Loss:  0.783, Val Acc: 74.16%, Time: 16.86s *
Epoch: 6, Train Loss: 0.4392, Train Acc: 84.46%, Val Loss: 0.7497, Val Acc: 75.67%, Time: 16.94s *
Epoch: 7, Train Loss: 0.3621, Train Acc: 87.41%, Val Loss: 0.6991, Val Acc: 78.19%, Time: 16.87s *
Epoch: 8, Train Loss: 0.3048, Train Acc: 89.38%, Val Loss: 0.7392, Val Acc: 77.35%, Time: 16.92s 
Epoch: 9, Train Loss: 0.2535, Train Acc: 91.28%, Val Loss: 0.7998, Val Acc: 77.52%, Time: 16.98s 
Epoch: 10, Train Loss: 0.2184, Train Acc: 92.62%, Val Loss: 0.8792, Val Acc: 78.19%, Time: 

Epoch: 1, Train Loss:  1.614, Train Acc: 33.02%, Val Loss:  1.374, Val Acc: 46.39%, Time: 42.65s *
Epoch: 2, Train Loss:  1.197, Train Acc: 53.63%, Val Loss:  1.071, Val Acc: 58.99%, Time: 40.33s *
Epoch: 3, Train Loss:  0.873, Train Acc: 67.63%, Val Loss: 0.8396, Val Acc: 67.39%, Time: 42.67s *
Epoch: 4, Train Loss: 0.6715, Train Acc: 75.89%, Val Loss:  0.745, Val Acc: 71.26%, Time: 39.72s *
Epoch: 5, Train Loss: 0.5448, Train Acc: 79.95%, Val Loss: 0.6975, Val Acc: 72.44%, Time: 42.74s *
Epoch: 6, Train Loss: 0.4414, Train Acc: 84.31%, Val Loss: 0.7223, Val Acc: 73.11%, Time: 39.56s *
Epoch: 7, Train Loss: 0.3653, Train Acc: 86.92%, Val Loss: 0.6948, Val Acc: 76.81%, Time: 42.71s *
Epoch: 8, Train Loss: 0.3073, Train Acc: 89.04%, Val Loss: 0.7774, Val Acc: 75.13%, Time: 39.35s 
Epoch: 9, Train Loss: 0.2586, Train Acc: 91.12%, Val Loss: 0.6772, Val Acc: 78.49%, Time: 42.31s *
Epoch: 10, Train Loss:  0.214, Train Acc: 92.54%, Val Loss: 0.7524, Val Acc: 76.64%, Time: 40.08s 
Epoch: 11, 

Epoch: 5, Train Loss: 0.5592, Train Acc: 80.37%, Val Loss: 0.6712, Val Acc: 76.47%, Time: 41.00s *
Epoch: 6, Train Loss:  0.449, Train Acc: 84.51%, Val Loss: 0.6773, Val Acc: 76.13%, Time: 41.81s 
Epoch: 7, Train Loss: 0.3804, Train Acc: 86.62%, Val Loss: 0.6676, Val Acc: 80.00%, Time: 41.43s *
Epoch: 8, Train Loss: 0.3064, Train Acc: 89.31%, Val Loss: 0.6626, Val Acc: 79.33%, Time: 41.97s 
Epoch: 9, Train Loss: 0.2557, Train Acc: 91.39%, Val Loss: 0.6846, Val Acc: 79.33%, Time: 40.12s 
Epoch: 10, Train Loss: 0.2254, Train Acc: 91.97%, Val Loss: 0.7663, Val Acc: 78.99%, Time: 41.57s 
Epoch: 11, Train Loss: 0.1801, Train Acc: 93.58%, Val Loss: 0.7396, Val Acc: 79.16%, Time: 40.27s 
No optimization for a long time, auto-stopping...
Precision, Recall and F1-Score...
              precision    recall  f1-score   support

        ABBR       1.00      0.75      0.86        12
        DESC       0.74      0.80      0.77       136
        ENTY       0.74      0.76      0.75       130
         

[0.790268456375839,
 0.7785234899328859,
 0.7478991596638656,
 0.7899159663865546,
 0.7546218487394958,
 0.7899159663865546,
 0.7596638655462185,
 0.7815126050420168,
 0.7915966386554621,
 0.7798319327731092]